In [1]:
import os,pickle,torch
from model import GPT

In [2]:
out_dir = 'long'
model_dir='long'
seed=1337
device="mps"
dtype="float16"
compile=False

In [3]:
torch.manual_seed(seed)

In [4]:
ckpt_path = os.path.join(model_dir, 'ckpt.pt')
checkpoint = torch.load(ckpt_path, map_location=device)
config=checkpoint["config"]
model = GPT(config)
state_dict = checkpoint['model']
unwanted_prefix = '_orig_mod.'
for k,v in list(state_dict.items()):
    if k.startswith(unwanted_prefix):
        state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
model.load_state_dict(state_dict)

number of parameters: 3.37M


<All keys matched successfully>

In [5]:
model.eval()
model.to(device)
if compile:
    model = torch.compile(model)

In [6]:
with open(os.path.join(out_dir, 'meta.pkl'), 'rb') as f:
    meta = pickle.load(f)
stoi, itos = meta['stoi'], meta['itos']
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [8]:
start="|"
num_samples=5
max_new_tokens=50
temperature=0.5
top_k=8

start_ids = encode(start)
x = (torch.tensor(start_ids, dtype=torch.long, device=device)[None, ...])

# run generation
with torch.no_grad():
    for k in range(num_samples):
        y = model.generate(x, max_new_tokens, temperature=temperature, top_k=top_k)
        print(decode(y[0].tolist()))

|roselinda|
|serena|
|mariadolores|
|mindo|
|mariacelina|
